In [ ]:
import numpy as np

##Metrics

### Correct detected Track - True Positive

**Temporal overlap :**  
 $TO(GT_{i},ST_{j}) = \frac{ Length( GT_{i}{\bigcap}ST_{j} )} {Length(GT_{i})}$

**Spatial overlap :**  
$A(GT_{i},ST_{j}) = \frac{ Area( GT_{i}{\bigcap}ST_{j} )} {Area( GT_{i}{\bigcup}ST_{j} )}$


In [ ]:
THRESHOLD_OVERLAP = 0.15
THRESHOLD_SPATIAL_OVERLAP = 0.20

def temporal_overlap_cdt(groundTruthBbox, systemTrackBbox) :
  '''
  Paramètres :
  groundTruthBbox : array qui contient (top_left_x, top_left_y, width, height)
  '''
  gt_box1, gt_box2 = groundTruthBbox[:2], groundTruthBbox[:2]+groundTruthBbox[2:]
  st_box1, st_box2 = systemTrackBbox[:2], systemTrackBbox[:2]+systemTrackBbox[2:]

  xA = max(gt_box1[0], st_box1[0])
  yA = max(gt_box1[1], gt_box1[1])
  xB = min(gt_box2[0], st_box2[0])
  yB = min(gt_box2[0], st_box2[1])

  temp_overlap = max(0, yB - yA) / groundTruthBbox[-1]

  return temp_overlap


def spatial_overlap(groundTruthBbox, systemTrackBbox) :
  gt_box1, gt_box2 = groundTruthBbox[:2], groundTruthBbox[:2]+groundTruthBbox[2:]
  st_box1, st_box2 = systemTrackBbox[:2], systemTrackBbox[:2]+systemTrackBbox[2:]

  xA = max(gt_box1[0], st_box1[0])
  yA = max(gt_box1[1], gt_box1[1])
  xB = min(gt_box2[0], st_box2[0])
  yB = min(gt_box2[0], st_box2[1])

  #the area of intersection rectangle
  inter_area = max(0, xB - xA) * max(0, yB - yA)
  #L'aire du rectangle gtbox
  gtbox_area = groundTruthBbox[2]*groundTruthBbox[3]
  #L'aire du rectangle stbox
  stbox_area = systemTrackBbox[2]*systemTrackBbox[3]

  union_area = gtbox_area + stbox_area - inter_area

  return inter_area / union_area

def spatial_overlap_2(gtBboxes, stBboxes) :
  '''
  Paramètres :
  groundTruthBbox : matrice des groundTruthBboxes où chaque ligne est un bbox
  '''
  spatial_overlap_sum = 0
  for j in range(len(gtBboxes)) :
    spatial_overlap_sum += spatial_overlap(gtBboxes[j], stBboxes[j])
  
  return spatial_overlap_sum / len(gtBboxes)

def cdtMetric(gtBboxes, stBboxes) :
  '''
  Paramètres :
  groundTruthBbox : matrice des groundTruthBboxes où chaque ligne est un bbox
  '''
  boolCdtMetric = True
  for j in range(len(gtBboxes)) :
    if temporal_overlap_cdt(gtBboxes[j], stBboxes[j]) < THRESHOLD_OVERLAP :
      return False
  if boolCdtMetric and (spatial_overlap_2(gtBboxes, stBboxes) > THRESHOLD_SPATIAL_OVERLAP) :
    return True
  else :
    return False

### False alarm track (FAT) of False Positive (FP):

In [ ]:
def temporal_overlap_fat(groundTruthBbox, systemTrackBbox) :
  '''
  Paramètres :
  groundTruthBbox : array qui contient (top_left_x, top_left_y, width, height)
  '''
  gt_box1, gt_box2 = groundTruthBbox[:2], groundTruthBbox[:2]+groundTruthBbox[2:]
  st_box1, st_box2 = systemTrackBbox[:2], systemTrackBbox[:2]+systemTrackBbox[2:]

  xA = max(gt_box1[0], st_box1[0])
  yA = max(gt_box1[1], gt_box1[1])
  xB = min(gt_box2[0], st_box2[0])
  yB = min(gt_box2[0], st_box2[1])

  temp_overlap = max(0, yB - yA) / systemTrackBbox[-1]

  return temp_overlap

def fatMetric(gtBboxes, stBboxes) :
  boolFatMetric = True
  for j in range(len(gtBboxes)) :
    if temporal_overlap_fat(gtBboxes[j], stBboxes[j]) > THRESHOLD_OVERLAP :
      boolFatMetric = False
  if boolFatMetric or (spatial_overlap_2(gtBboxes, stBboxes) < THRESHOLD_SPATIAL_OVERLAP) :
    return True
  else :
    return False

### Track detection failure (TDF):

In [ ]:
def tdfMetric(gtBboxes, stBboxes) :
  boolFatMetric = True
  for j in range(len(gtBboxes)) :
    if temporal_overlap_cdt(gtBboxes[j], stBboxes[j]) > THRESHOLD_OVERLAP :
      boolFatMetric = False
  if boolFatMetric or (spatial_overlap_2(gtBboxes, stBboxes) < THRESHOLD_SPATIAL_OVERLAP) :
    return True
  else :
    return False

### Latency of the system track (LT):

$LT = start{\,}frame{\,}of{\,}ST_{j} − start{\,}frame{\,}of{\,}GT_{i}$

$\textbf{LT}$ estime le temps de difference entre la première frame du système de tracking (deep sort dans notre cas) et la première frame du GT (sans bounding boxes).
La latence optimal est zéro.
Plus la latence est proche de zéro plus le système du tracking est efficace.



